# Query WikiData

In [1]:
import requests
import pandas as pd
from collections import OrderedDict

In [2]:
url = 'https://query.wikidata.org/sparql'

## Building the Query - Essentials for each query

`?event wdt:P31/wdt:P279* wd:Q198.`
Event is an instance of (P31) or subclass of (P279) war (Q198).

`?event wdt:P710 ?participants.`
Event has participants (P710) Participant.

`?event wdt:P580 ?startDate.`
Event has start time (P580) StartDate.

`FILTER(YEAR(?startDate) >= 1946).`
Event has start date after 1946

## Wars & their participants

In [3]:
query1 = """
SELECT DISTINCT
  ?event ?eventLabel ?participant ?participantLabel
WHERE {
  ?event wdt:P31/wdt:P279* wd:Q198.
  OPTIONAL { ?event wdt:P710 ?participant }
  ?event wdt:P580 ?startDate
  FILTER(YEAR(?startDate) >= 1946).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

In [4]:
r1 = requests.get(url, params = {'format': 'json', 'query': query1})
data1 = r1.json()

wars_pars = []
for item in data1['results']['bindings']:
    wars_pars.append(OrderedDict({
        label : item[label]['value'] if label in item else None
        for label in ['event', 'eventLabel', 'participant', 'participantLabel']}))

In [5]:
df1 = pd.DataFrame(wars_pars)
df1['warID'] = df1['event'].str.split('/').str[-1]
df1['participantID'] = df1['participant'].str.split('/').str[-1]

In [6]:
df1

,event,eventLabel,participant,participantLabel,warID,participantID
0,http://www.wikidata.org/entity/Q190758,Darfur conflict,http://www.wikidata.org/entity/Q384535,African Union – United Nations Hybrid Operatio...,Q190758,Q384535
1,http://www.wikidata.org/entity/Q322607,War of Attrition,http://www.wikidata.org/entity/Q801,Israel,Q322607,Q801
2,http://www.wikidata.org/entity/Q322607,War of Attrition,http://www.wikidata.org/entity/Q241,Cuba,Q322607,Q241
3,http://www.wikidata.org/entity/Q322607,War of Attrition,http://www.wikidata.org/entity/Q810,Jordan,Q322607,Q810
4,http://www.wikidata.org/entity/Q303801,Lord's Resistance Army insurgency,http://www.wikidata.org/entity/Q1036,Uganda,Q303801,Q1036
...,...,...,...,...,...,...
847,http://www.wikidata.org/entity/Q18920712,War in Afghanistan (2015–),http://www.wikidata.org/entity/Q42418,Taliban,Q18920712,Q42418
848,http://www.wikidata.org/entity/Q18920712,War in Afghanistan (2015–),http://www.wikidata.org/entity/Q145,United Kingdom,Q18920712,Q145
849,http://www.wikidata.org/entity/Q11154636,Eelam War III,http://www.wikidata.org/entity/Q854,Sri Lanka,Q11154636,Q854
850,http://www.wikidata.org/entity/Q18920712,War in Afghanistan (2015–),http://www.wikidata.org/entity/Q30,United States of America,Q18920712,Q30


In [7]:
df1['warID'].nunique()

195

In [8]:
df1['participantID'].nunique()

495

In [24]:
df1.to_csv("Data/wars_participants.csv", index=False, encoding='utf-8')

## Wars

In [9]:
query2 = """
SELECT DISTINCT
  ?event ?eventLabel ?eventDescription ?startDate ?endDate ?location ?locationLabel ?casualties ?prevWar ?nextWar ?largerConflict ?largerConflictLabel ?largerConflictDescription
WHERE {
  ?event wdt:P31/wdt:P279* wd:Q198.
  ?event wdt:P580 ?startDate.
  OPTIONAL { ?event wdt:P582 ?endDate. }
  OPTIONAL { ?event wdt:P276 ?location. }
  OPTIONAL { ?event wdt:P155 ?prevWar. }
  OPTIONAL { ?event wdt:P156 ?nextWar. }
  OPTIONAL { ?event wdt:P361 ?largerConflict. }
  OPTIONAL { ?event wdt:P1120 ?casualties }
  FILTER(YEAR(?startDate) >= 1946).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

In [10]:
r2 = requests.get(url, params = {'format': 'json', 'query': query2})
data2 = r2.json()

wars = []
for item in data2['results']['bindings']:
    wars.append(OrderedDict({
        label : item[label]['value'] if label in item else None
        for label in ['event', 'eventLabel', 'eventDescription', 'startDate', 'endDate', 'location', 'locationLabel', 
                      'casualties', 'prevWar', 'nextWar', 'largerConflict']}))

In [11]:
df2 = pd.DataFrame(wars)
df2['warID'] = df2['event'].str.split('/').str[-1]
df2['locationID'] = df2['location'].str.split('/').str[-1]
df2['prevWarID'] = df2['prevWar'].str.split('/').str[-1]
df2['nextWarID'] = df2['nextWar'].str.split('/').str[-1]
df2['largerConflictID'] = df2['largerConflict'].str.split('/').str[-1]

In [12]:
df2

,event,eventLabel,eventDescription,startDate,endDate,location,locationLabel,casualties,prevWar,nextWar,largerConflict,warID,locationID,prevWarID,nextWarID,largerConflictID
0,http://www.wikidata.org/entity/Q170682,Gaza War,December 2008-January 2009 armed conflict in t...,2008-12-27T00:00:00Z,2009-01-18T00:00:00Z,http://www.wikidata.org/entity/Q188781,Southern District,None,None,None,http://www.wikidata.org/entity/Q553184,Q170682,Q188781,None,None,Q553184
1,http://www.wikidata.org/entity/Q181533,Bosnian War,international armed conflict that took place i...,1992-04-06T00:00:00Z,1995-12-14T00:00:00Z,http://www.wikidata.org/entity/Q188736,Bosnia,None,None,None,http://www.wikidata.org/entity/Q242352,Q181533,Q188736,None,None,Q242352
2,http://www.wikidata.org/entity/Q48249,Falklands War,1982 war between Argentina and the United Kingdom,1982-04-02T00:00:00Z,1982-06-14T00:00:00Z,http://www.wikidata.org/entity/Q35086,South Georgia and the South Sandwich Islands,891,None,None,http://www.wikidata.org/entity/Q4379537,Q48249,Q35086,None,None,Q4379537
3,http://www.wikidata.org/entity/Q170682,Gaza War,December 2008-January 2009 armed conflict in t...,2008-12-27T00:00:00Z,2009-01-18T00:00:00Z,http://www.wikidata.org/entity/Q188781,Southern District,None,None,None,http://www.wikidata.org/entity/Q15059994,Q170682,Q188781,None,None,Q15059994
4,http://www.wikidata.org/entity/Q179275,War in North-West Pakistan,armed conflict involving Pakistan and armed mi...,2004-03-16T00:00:00Z,None,http://www.wikidata.org/entity/Q208270,Federally Administered Tribal Areas,None,None,None,http://www.wikidata.org/entity/Q185729,Q179275,Q208270,None,None,Q185729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,http://www.wikidata.org/entity/Q2634899,Q2634899,None,1989-01-01T00:00:00Z,2001-01-01T00:00:00Z,None,None,None,None,None,None,Q2634899,None,None,None,None
310,http://www.wikidata.org/entity/Q16911838,Libyan civil war (2014–present),conflict among rival groups seeking to control...,2014-05-16T00:00:00Z,None,http://www.wikidata.org/entity/Q1016,Libya,None,None,None,http://www.wikidata.org/entity/Q33761,Q16911838,Q1016,None,None,Q33761
311,http://www.wikidata.org/entity/Q17324420,2014 Israel–Gaza conflict,War fought between the Israel Defense Forces a...,2014-07-08T00:00:00Z,2014-08-26T00:00:00Z,http://www.wikidata.org/entity/Q39760,Gaza Strip,2205,None,None,http://www.wikidata.org/entity/Q553184,Q17324420,Q39760,None,None,Q553184
312,http://www.wikidata.org/entity/Q2444955,Rhodesian Bush War,civil conflict in Southern Africa from 1964 to...,1964-07-04T00:00:00Z,1979-12-12T00:00:00Z,http://www.wikidata.org/entity/Q217169,Rhodesia,None,None,None,http://www.wikidata.org/entity/Q1146918,Q2444955,Q217169,None,None,Q1146918


In [25]:
df2.to_csv("Data/wars.csv", index=False, encoding='utf-8')

## Participants

sovereign states

In [13]:
query3 = """
SELECT DISTINCT
  ?participant ?participantLabel ?participantDescription ?countryCode ?inception ?dissolved ?government ?governmentLabel
WHERE {
  ?event wdt:P31/wdt:P279* wd:Q198.
  ?event wdt:P710 ?participant.
  ?participant wdt:P31 ?participantType.
  OPTIONAL { ?participant wdt:P298 ?countryCode }.
  ?participant wdt:P31/wdt:P279* wd:Q3624078.
  OPTIONAL { ?participant wdt:P571 ?inception. }
  OPTIONAL { ?participant wdt:P576 ?dissolved. }
  OPTIONAL { ?participant wdt:P122 ?government. }
  ?event wdt:P580 ?startDate.
  OPTIONAL { ?event wdt:P582 ?endDate }.
  FILTER(YEAR(?startDate) >= 1946).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

In [14]:
r3 = requests.get(url, params = {'format': 'json', 'query': query3})
data3 = r3.json()

states = []
for item in data3['results']['bindings']:
    states.append(OrderedDict({
        label : item[label]['value'] if label in item else None
        for label in ['participant', 'participantLabel', 'participantDescription', 'countryCode', 'inception', 'dissolved', 
                      'government', 'governmentLabel']}))

In [15]:
df3 = pd.DataFrame(states)
df3['participantID'] = df3['participant'].str.split('/').str[-1]
df3['govtID'] = df3['government'].str.split('/').str[-1]

In [16]:
df3

,participant,participantLabel,participantDescription,countryCode,inception,dissolved,government,governmentLabel,participantID,govtID
0,http://www.wikidata.org/entity/Q142,France,sovereign state with mainland in Europe and se...,FRA,None,None,http://www.wikidata.org/entity/Q49890,semi-presidential system,Q142,Q49890
1,http://www.wikidata.org/entity/Q142,France,sovereign state with mainland in Europe and se...,FRA,None,None,http://www.wikidata.org/entity/Q179164,unitary state,Q142,Q179164
2,http://www.wikidata.org/entity/Q142,France,sovereign state with mainland in Europe and se...,FRA,None,None,http://www.wikidata.org/entity/Q188961,decentralization,Q142,Q188961
3,http://www.wikidata.org/entity/Q30,United States of America,country in North America,USA,1776-07-04T00:00:00Z,None,http://www.wikidata.org/entity/Q49892,presidential system,Q30,Q49892
4,http://www.wikidata.org/entity/Q145,United Kingdom,"sovereign state in the British Isles, off the ...",GBR,1927-04-12T00:00:00Z,None,http://www.wikidata.org/entity/Q41614,constitutional monarchy,Q145,Q41614
...,...,...,...,...,...,...,...,...,...,...
156,http://www.wikidata.org/entity/Q833,Malaysia,sovereign state in Southeast Asia,MYS,1957-01-01T00:00:00Z,None,http://www.wikidata.org/entity/Q41614,constitutional monarchy,Q833,Q41614
157,http://www.wikidata.org/entity/Q963,Botswana,sovereign state in Southern Africa,BWA,1966-01-01T00:00:00Z,None,http://www.wikidata.org/entity/Q7270,republic,Q963,Q7270
158,http://www.wikidata.org/entity/Q836,Myanmar,sovereign state in Southeast Asia,MMR,1948-01-04T00:00:00Z,None,http://www.wikidata.org/entity/Q7270,republic,Q836,Q7270
159,http://www.wikidata.org/entity/Q965,Burkina Faso,sovereign state in Africa,BFA,1960-01-01T00:00:00Z,None,http://www.wikidata.org/entity/Q7270,republic,Q965,Q7270


In [26]:
df3.to_csv("Data/participants_states.csv", index=False, encoding='utf-8')

armed organizations

In [19]:
query4 = """
SELECT DISTINCT
  ?participant ?participantLabel ?participantDescription ?inception ?dissolved ?type ?typeLabel ?typeDescription
WHERE {
  ?event wdt:P31/wdt:P279* wd:Q198.
  ?event wdt:P710 ?participant.
  ?participant wdt:P31 ?participantType.
  ?participant wdt:P31/wdt:P279* wd:Q17149090.
  OPTIONAL { ?participant wdt:P571 ?inception. }
  OPTIONAL { ?participant wdt:P576 ?dissolved. }
  ?participant wdt:P31 ?type.
  ?event wdt:P580 ?startDate.
  OPTIONAL { ?event wdt:P582 ?endDate }.
  FILTER(YEAR(?startDate) >= 1946).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""

In [20]:
r4 = requests.get(url, params = {'format': 'json', 'query': query4})
data4 = r4.json()

orgs = []
for item in data4['results']['bindings']:
    orgs.append(OrderedDict({
        label : item[label]['value'] if label in item else None
        for label in ['participant', 'participantLabel', 'participantDescription', 'inception', 'dissolved', 
                      'type', 'typeLabel', 'typeDescription']}))

In [21]:
df4 = pd.DataFrame(orgs)
df4['participantID'] = df4['participant'].str.split('/').str[-1]
df4['typeID'] = df4['type'].str.split('/').str[-1]

In [22]:
df4

,participant,participantLabel,participantDescription,inception,dissolved,type,typeLabel,typeDescription,participantID,typeID
0,http://www.wikidata.org/entity/Q80312,Liberation Tigers of Tamil Eelam,militant organization in Sri Lanka,1976-01-01T00:00:00Z,2009-01-01T00:00:00Z,http://www.wikidata.org/entity/Q7278,political party,organization that seeks to influence governmen...,Q80312,Q7278
1,http://www.wikidata.org/entity/Q80312,Liberation Tigers of Tamil Eelam,militant organization in Sri Lanka,1976-01-01T00:00:00Z,2009-01-01T00:00:00Z,http://www.wikidata.org/entity/Q20642011,guerrilla movement,military organization practicing irregular war...,Q80312,Q20642011
2,http://www.wikidata.org/entity/Q58967,Israel Defense Forces,combined military forces of Israel,1948-05-26T00:00:00Z,None,http://www.wikidata.org/entity/Q772547,armed forces,collective organization of a country's armed m...,Q58967,Q772547
3,http://www.wikidata.org/entity/Q160805,United Nations Interim Force in Lebanon,None,1978-01-01T00:00:00Z,None,http://www.wikidata.org/entity/Q1456930,peacekeeping force,military unit of one or more uninvolved countr...,Q160805,Q1456930
4,http://www.wikidata.org/entity/Q7184,NATO,intergovernmental military alliance of Western...,1949-04-04T00:00:00Z,None,http://www.wikidata.org/entity/Q245065,intergovernmental organization,organization composed primarily of sovereign s...,Q7184,Q245065
...,...,...,...,...,...,...,...,...,...,...
107,http://www.wikidata.org/entity/Q27628118,Internal Troops of Georgia,None,1991-01-01T00:00:00Z,None,http://www.wikidata.org/entity/Q176799,military unit,organization formed as part of an armed force,Q27628118,Q176799
108,http://www.wikidata.org/entity/Q252054,Haganah,A Jewish paramilitary organization in what was...,1920-06-15T00:00:00Z,1948-05-28T00:00:00Z,http://www.wikidata.org/entity/Q207320,paramilitary,"organized force, possibly armed, other than th...",Q252054,Q207320
109,http://www.wikidata.org/entity/Q3374306,Sudan Revolutionary Front,None,None,None,http://www.wikidata.org/entity/Q7210356,political organisation,group involved in political activities,Q3374306,Q7210356
110,http://www.wikidata.org/entity/Q755973,Interahamwe,paramilitary group involved in 1994 Rwandan Ge...,1992-01-01T00:00:00Z,1994-01-01T00:00:00Z,http://www.wikidata.org/entity/Q207320,paramilitary,"organized force, possibly armed, other than th...",Q755973,Q207320


In [27]:
df4.to_csv("Data/participants_armedorgs.csv", index=False, encoding='utf-8')